In [9]:
import collections

目录: 1.字符串 2.数组 3.链表 4.哈希表 5.栈 6.队列 7.树 8.堆 9.前缀树(字典树) 10.图

## 1. 字符串

- 双指针
    * 关键字: 变位词, 连续子串, 不含重复字符最长子串, 含有目标串所有字符的最短子串, 回文串
    * 变位词: 长度相同, 字符种类数量相同，但排列可能不同。所以可用固定长度滑动窗口+哈希表(或26位数组)
    * 不含重复字符最长最长子串: 用哈希表记录滑动窗口中字符的数量，移动右指针直到满足条件，再移动左指针不断更新res
    * 回文串: 前后两个指针，while left < right可以不用管奇数位还是偶数位
    
- 中心拓展法
    * 关键字: 回文子串的个数
    * 暴力法: 列举所有子串，并判断每个字串是否是回文串
    * 中心拓展法: 以当前元素为中心，双指针向左右拓展; 或以当前元素和下一个元素为中心，双指针向左右拓展
    

常见题型: 字符中的变位词，不含重复字符的最长连续子串，含目标串所有字符的最短子串，是否是回文串，回文子串个数，包含某个元音的子字符串个数

In [6]:
# 字符串中的变位词, s2中是否含有s1的排列
def checkInclusion(s1, s2):
    # 因为s1的长度是固定的，就是s2中同样长度的子串是否有相同的哈希表示(字符种类和个数相同)
    # 双指针
    n1, n2 = len(s1), len(s2)
    if n1 > n2: return False
    d1, d2 = [0] * 26, [0] * 26
    for c in s1:
        d1[ord(c) - 97] += 1
    left = 0
    for right in range(n2):
        if d1 == d2:
            return True
        if right - left == n1:
            d2[ord(s2[left]) - 97] -= 1
            left += 1
        d2[ord(s2[right]) - 97] += 1
    # 注意这里！！！如果res相关的是在循环开始判断的，而判断之后还有新操作，那么循环结束后还需要再判断一次
    return d1 == d2
    
    # 更妙的双指针
    n1, n2 = len(s1), len(s2)
    if n1 > n2: return False
    d1, d2 = [0] * 26, [0] * 26
    for i in range(n1):
        d1[ord(s1[i]) - 97] += 1
        d2[ord(s2[i]) - 97] += 1
    for i in range(n1, n2):
        if d1 == d2: 
            return True
        d2[ord(s2[i-n1]) - 97] -= 1
        d2[ord(s2[i]) - 97] += 1
    return d1 == d2
print("checkInclusion: ", checkInclusion("ab", "eidbaooo"))

# 不含重复字符的最长子串(输入包括字母、数字、符号、空格)
def lengthOfLongestSubstring(s):
    # 用哈希表记录滑动窗口中元素的个数，如果超过两个移动左指针直至变回一个
    res = 0
    left = 0
    cnt = collections.defaultdict(int)
    for right in range(len(s)):
        cnt[s[right]] += 1
        # 如果当前元素数量是是1说明滑动窗口是满足条件的
        if cnt[s[right]] == 1:
            res = max(res, right - left + 1)
        else:
            while cnt[s[right]] > 1:
                cnt[s[left]] -= 1
                left += 1
    return res
print("lengthOfLongestSubstring: ", lengthOfLongestSubstring("bbbbb"))

# 含有目标串t所有字符的s最短子串，输入为英文字母
def minWindow(s, t):
    # 先移动右指针直到满足条件，再不断减小左指针，并更新res，直到不满足条件
    # 条件是: 滑动窗口中包含t的各个字母，数量要 ≥ t中对应数量
    if len(s) < len(t): return ""
    res = ""
    cnt_s, cnt_t = collections.defaultdict(int), collections.defaultdict(int)
    # 先记录t串单词种类及数量
    for c in t:
        cnt_t[c] += 1
    # 然后开始滑动窗口，注意target表示t中元素种类数，必须要去重
    cnt, left = 0, 0
    target = len(set(t))
    for right in range(len(s)):
        cnt_s[s[right]] += 1
        # 如果数量相同说明这个字母达到条件了
        if cnt_s[s[right]] == cnt_t[s[right]]:
            cnt += 1
            while cnt == target:
                # 如果满足条件的子串更短，那么更新res，注意左右指针是前闭后闭
                if not res or (right - left + 1) < len(res):
                    res = s[left: right+1]
                # 移动左指针，直到不满足条件
                cnt_s[s[left]] -= 1
                if cnt_s[s[left]] < cnt_t[s[left]]:
                    cnt -= 1
                left += 1
    return res
print("minWindow: ", minWindow("ADOBECODEBANC", "ABC"))

# 回文子串个数。输入为小写字母
def countSubStrings(s):
    # 暴力列举所有子串 / 中心拓展法
    def center(left, right):
        res = 0
        while 0 <= left and right < len(s) and s[left] == s[right]:
            res += 1
            left -= 1
            right += 1
        return res
    res = 0
    for i in range(len(s)):
        res += center(i, i)
        res += center(i, i+1)
    return res
print("countSubStrings: ", countSubStrings("aaa"))

# 包含某个字符的子字符串数量
def countVowels(word):
    # 统计包含某个字符的连续子串个数: 子串可以写成word[l, r]左闭右闭，那么l的取值范围是[0, i]，r的取值范围是[i, n-1]
    res = 0
    n = len(word)
    for i in range(n):
        if word[i] in ["a", "e", "i", "o", "u"]:
            res += (i + 1) * (n - i)
    return res
print("countVowels: ", countVowels("noosabasboosa"))


checkInclusion:  True
lengthOfLongestSubstring:  1
minWindow:  BANC
countSubStrings:  6
countVowels:  237


## 2. 数组

- 双指针
    * 关键字: 有序数组，连续子数组，子数组和小于target
    * 条件判断经常是: 先移动右指针right<len(nums)直到不满足条件，再移动左指针left<right逐个判断
- 前缀和/积
    * 关键字: 连续子数组，最长连续子数组，子数组和小于target
    * 如果数组中有正数和负数的话，就不能使用双指针，因为没有办法判定移动左指针还是右指针，就只能用前缀和了！！
    * 如果要找子数组和==target，可以利用哈希表加快查找速度

常见题型: 数组中和为0的两个数/三个数，和大于等于K的最短子数组，乘积小于K的子数组，和为K的子数组，0/1个数相同的子数组，二维子矩阵和

In [7]:
# 数组中和为0的三个数(不重复的三元组)
def threeSum(nums):
    # 先排序，第一个数必须为负数或0。锁定第一个数之后用双指针来找后两个数的和是否存在
    # 不重复: 说明第一个数字每次必须是不同的
    nums.sort()
    res = []
    for i in range(len(nums)):
        if nums[i] > 0: break
        if i > 0 and nums[i] == nums[i-1]: continue
        j, k = i + 1, len(nums) - 1
        while j < k:
            if nums[j] + nums[k] == -nums[i]:
                res.append(list([nums[i], nums[j], nums[k]]))
                # 一定要找到下一个不一样的元素才继续循环
                while j < k and nums[j] + nums[k] == -nums[i]:
                    k -= 1
            elif nums[j] + nums[k] > -nums[i]:
                k -= 1
            else:
                j += 1
    return res
print("threeSum: ", threeSum([-1,0,1,2,-1,-4]))

# 和大于等于target的最短子数组长度
def minSubArrayLen(target, nums):
    # 双指针，先找到满足条件的数组，再逐渐移动左指针直到不满足条件
    left, right = 0, 0
    total = 0
    res = len(nums) + 1
    while right < len(nums):
        while right < len(nums) and total < target:
            total += nums[right]
            right += 1
        # 出来之后total>=target
        while left < right and total >= target:
            res = min(res, right - left)
            total -= nums[left]
            left += 1
        # 出来之后
    return 0 if res == len(nums) + 1 else res
print("minSubArrayLen:", minSubArrayLen(7, [2,3,1,2,4,3]))
            
# 乘积小于K的子数组数目
def numSubarrayProductLessThanK(nums, K):
    # 输入都为正数
    # 两种方法: 
    # 1. 前缀积。看前面有多少个数满足 <= 当前积 / K，与二分查找连用。(会超时, 取决于样本数量)
    # 2. 双指针。一步一步移动右指针，统计以当前右指针为右边界的数组有多少个满足条件的
    #          (左指针移动到滑动窗口乘积 < K) 此时数量为(right - left + 1)
    res = 0
    left, temp = 0, 1
    for right in range(len(nums)):
        temp *= nums[right]
        while left < right and temp >= K:
            temp /= nums[left]
            left += 1
        res += (right - left + 1)
    return res
print("numSubarrayProductLessThanK: ", numSubarrayProductLessThanK([10,5,2,6], 100))
           
# 和为K的子数组
def subArraySum(nums, K):
    # 不能用双指针！！！！！！因为数组中有正数有负数，没有办法判定移动左指针还是右指针
    # 换前缀和
    temp, res = 0, 0
    dic = collections.defaultdict(int)
    dic[0] = 1
    for right in range(len(nums)):
        temp += nums[right]
        res += (dic[temp-K])
        dic[temp] += 1
    return res
print("subArraySum: ", subArraySum([1,2,3], 3))
            
# 0和1个数相同的子数组
def findMaxLength(nums):
    # 记录下1比0多的个数，如果之前出现 *相同* 的说明可以抵消
    # 注意判断temp为0时，最长的是从它到前面所有元素
    res = 0
    temp, dic = 0, {}
    dic[0] = -1
    for i in range(len(nums)):
        temp = temp + 1 if nums[i] == 1 else temp - 1
        # 这里判断条件不能设置为0，因为序号可能为0！！
        if dic.get(temp) != None:
            res = max(res, i - dic.get(temp))
        else:
            dic[temp] = i
    return res
print("findMaxLength: ", findMaxLength([0, 1]))

# 二维子矩阵的和
def sumRegion(row1, col1, row2, col2):
    # 前缀和，统一以原点作为左上角点，右下角点可以确定一个矩形。
    # 那么由给定点组成的矩形 R = R(row2, col2) - R(row1, col2) - R(row2, col1) + R(row1, col1)
    pass

threeSum:  [[-1, -1, 2], [-1, 0, 1]]
minSubArrayLen: 2
numSubarrayProductLessThanK:  8
subArraySum:  2
findMaxLength:  2


## 3. 链表

- 用while判断时记得移动链表
- 快慢指针求中点、入口等
- 用栈可以反转，哈希表可以去重
- 反转某一段: 双指针，左指针指向反转区间的前一项，右指针指向反转区间的最后一项，用temp保存后面的数组。左指针一开始要指向新节点，避免头结点是反转区间结点。

常见题型: 求中点，删除倒数第K个节点，环的入口节点，两个链表的第一个公共节点，反转链表，链表相加，重排链表，回文链表，循环链表

In [ ]:
# 链表结构
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = None

# 删除倒数第N个节点 ※
def removeNthFromEnd(head, n):
    # 三种方法:
    # 1.双指针: 第一个先走n步，第二个指向一个新节点：两者同时出发，第一个到末尾了，第二个指向的就是待删除的前一项
    # 2.计算链表长度: 先遍历一遍，统计节点数量，再从头走total-n到待删除的前一项
    # 3.栈: 先依次入栈，弹出的时候再删除第n个
    res = ListNode(0, head)
    first = head
    second = res
    for _ in range(n):
        first = first.next
    while first:
        first = first.next
        second = second.next
    # 此时second再待删除节点前一项
    second.next = second.next.next
    return second.next


# 环的入口节点
def detectCycle(head):
    # 两种方法:
    # 1.哈希表: 记录下访问过的节点，如果再次访问就说明循环了；如果到None说明没有循环
    # 2.快慢指针: 令快指针是慢指针速度两倍，设head到环起始点距离为a，环起始点到第一次相遇点距离为b，第一次相遇点到环起始点距离为c，
    #            则第一次相遇有 a+k(b+c)+b = 2*(a+b)，其中k为多走的圈数；化简可得 a = (k-1)(b+c)+c。因此让快指针回到起点，
    #            两个指针一步步地走，再一次相遇点就是环起始点。(从头走到环起始点恰好等于从相遇点走到起始点)
    fast, slow = head, head
    # 判断条件是fast能不能继续前进，如果遇到None说明无环
    while fast and fast.next:
        slow = slow.next
        fast = fast.next.next
        if slow == fast:
            fast = head
            while fast != slow:
                slow, fast = slow.next, fast.next
            return slow
    return None


# 相交链表
def getInterSectionNode(headA, headB):
    # 首先让两个指针指向两个链表，保证两个走过的总长度一样即可。一个指针遍历完了去另一个链表的头结点，最后两者要么相交要么都为None
    tempA, tempB = headA, headB
    while tempA != tempB:
        tempA = tempA.next if tempA else headB
        tempB = tempB.next if tempB else headA
    return tempA


# 反转链表 ※
def reverseList(head):
    # 让结果指向空，先保存下一个节点，然后每来一个就指向结果，结果再左移，经典四件套
    res = None
    while head:
        temp = head.next
        head.next = res
        res = head
        head = temp
    return res

# 两数相加，两个链表表示两个数，求他们的和
def addTwoNumbers(headA, headB):
    # 两种方法:
    # 1.反转求和: 两个都反转保证末尾对齐，再逐个考虑进位
    # 2.用栈储存: 先遍历，再逐个弹出

# 反转某一特定区间。[m, n]前闭后闭, m从0开始
def reverseSpecifiedInterval(head, m, n):
    # 双指针。前一个指针指向待反转区间的前一项，后一个指针指向待反转区间的最后一项
    # 要在头节点前面加一个新节点，保证头节点也能作为待反转节点
    first = LinkNode()
    first.next = head
    second = head
    res = first
    for _ in range(m):
        first = first.next
    for _ in range(n):
        second = second.next
    # 断开反转区间和后面的，并暂存后面的
    temp = second.next
    second.next = None
    first.next = reverse(first.next)
    # 反转后再将后面的接回来
    for _ in range(n - m + 1):
        first = first.next
    first.next = temp
    return res.next
    
    
    

## 4. 哈希表

- O(1)时间的插入、修改、删除；
- 与顺序无关与数量有关的统计问题；
- 只有小写字母还可以用26长度的数组模拟哈希表，"a"的ASCII为97

常见问题: 设计最少使用缓存LRU，变位词，变位词组 (以不变量作为key: 包含26个字母的情况)，日程表

In [2]:
class LinkNode:
    # 用双向链表来储存，key就是key，val存的是该key对应的链表节点地址(如果存的是val的话会超时)
    # 如果dic存的是val而不是节点地址，则需要遍历才能找到该节点
    def __init__(self, key = -1, val = -1):
        self.key = key
        self.val = val
        self.next = None
        self.prev = None

class LRUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cnt = 0
        self.dic = {}
        self.head = None

    def get(self, key: int) -> int:
        # 如果不存在则返回-1就好了
        if self.dic.get(key, -1) == -1:
            return -1
        # 如果该节点就在链表尾部
        if self.tail.key == key: 
            return self.dic.get(key).val
        # 如果该节点在链表头部，则需要将它移至尾部(head没有prev所以需要单独处理)
        if self.head.key == key:
            temp = self.head
            self.head = temp.next
            self.head.prev, temp.next = None, None
            self.tail.next = temp
            temp.prev = self.tail
            self.tail = temp
            return self.dic.get(key).val
        # 如果存在链表中部，需要将节点移至链表尾部
        temp = self.dic[key]
        temp.prev.next = temp.next
        temp.next.prev = temp.prev
        self.tail.next = temp
        temp.prev = self.tail
        self.tail = temp
        return self.dic.get(key).val

    def put(self, key: int, value: int) -> None:
        # 如果已经存在于dic中, 那么更新val，并把该节点移到最后面(调用get就可以了)
        if self.dic.get(key, -1) != -1:
            self.dic[key].val = value
            self.get(key)
        # 如果关键词不存在，则需要插入
        else:
            # 如果还没有元素, 那么当前的就是头元素和尾元素
            if not self.head:
                self.head = LinkNode(key, value)
                self.tail = self.head
                self.cnt += 1
                self.dic[key] = self.head
            # 如果没达到上限，那么需要插入新节点在最后，并更新数量以及dic
            elif self.cnt < self.capacity:
                self.tail.next = LinkNode(key, value)
                self.tail.next.prev = self.tail
                self.tail = self.tail.next
                self.cnt += 1
                self.dic[key] = self.tail
            # 如果达到上限了，那么需要删除链表最前面的节点，并将其dic数值改为-1
            else:
                self.dic[self.head.key] = -1
                self.head = self.head.next
                self.tail.next = LinkNode(key, value)
                self.tail.next.prev = self.tail
                self.tail = self.tail.next
                self.dic[key] = self.tail

# 日程表。给定开始时间和结束时间，判断一个新的日程能否加入               
# 要想加入，那么新日程的开始时间必须晚于之前日程的最晚结束时间。
# 如果新日程的开始时间小于之前日程的结束时间，但它的开始时间也小于之前日程的开始时间，也是可以插入的
# 因此就按顺序储存已有的结束时间，找到 ≥ 新日程开始时间的第一个结束时间；不存在的话直接加入，存在的话比较该结束时间对应的开始时间
from sortedcontainers import SortedDict
class MyCalendar:
    def __init__(self):
        self.dic = SortedDict()

    def book(self, start: int, end: int) -> bool:
        # 在已有的end中找到大于等于start的下标
        index = self.dic.bisect_right(start)
        # 如果存在这个end的值大于等于当前start
        if 0 <= index < len(self.dic):
            # 就比较这个end值对应的start，如果他小于当前end，说明会冲突
            if self.dic.values()[index] < end:
                return False
        # 否则就可以加入
        self.dic[end] = start
        return True


## 5. 栈

后进先出, 单调栈

常见题型: 后缀表达式，直方图最大矩形面积，矩阵中最大的矩阵

In [8]:
# 求后缀表达式(逆波兰表达式)
def evalRPN(tokens):
    # 遇到数字就进栈，遇到运算符就弹出两个处理再入栈，注意python对负数除法的实现
    res = []
    operations = {"+": lambda x, y: x + y,
                  "-": lambda x, y: x - y,
                  "*": lambda x, y: x * y,
                  "/": lambda x, y: int(x / y)}
    for token in tokens:
        if token not in operations.keys():
            res.append(int(token))
        else:
            y = res.pop()
            x = res.pop()
            res.append(operations[token](x, y))
    return res[0]
            
tokens = ["10","6","9","3","+","-11","*","/","*","17","+","5","+"]
print("evalRPN: ", evalRPN(tokens))


# 直方图最大矩形面积
def largestRectangleArea(heights):
    # 单调栈。中心思想就是记录下以每个节点为顶的最大矩形面积。
    # 要求最大矩形面积，矩形的宽就等于(左边比他矮，右边比他矮)这个区间(前开后开)。
    # 如果左边不存在比他矮的，说明这个高度可以一直加到最左边(-1)；如果右边都比他高，说明这个高度可以一直加到最右边(len(heights))
    # 保持单调递增的栈，如果出现比栈顶元素小的，说明找到了栈顶元素能形成的最大矩形面积
    # 栈中存的是下标！
    stack = [-1]
    res = 0
    for i in range(len(heights)):
            # 把i之前存在左右小于它高度的节点计算出来, 第一个节点不能弹出
        while stack[-1] != -1 and heights[i] < heights[stack[-1]]:
            temp = stack.pop()
            res = max(res, heights[temp] * (i - stack[-1] - 1))
        stack.append(i)
    # 最后再把栈中未弹出的节点计算。
    while stack[-1] != -1:
        temp = stack.pop()
        res = max(res, heights[temp] * (len(heights) - stack[-1] - 1))
    return res
    
heights = [2,1,5,6,2,3]
print("largestRectangleArea: ", largestRectangleArea(heights))

evalRPN:  22
largestRectangleArea:  10


## 6. 队列
先进先出，collections.deque()

常见题型: 二叉树的宽度优先搜索(bfs)，树的每层最大值/每层最左值/每层最右值，滑动窗口平均值

In [ ]:
def bfs(root):
    if not root: return
    res = []
    queue = collections.deque()
    queue.append(root):
    while queue:
        # 每层开始的地方
        temp = []
        # 限定这一层节点的数量
        for _ in range(len(queue)):
            # 先进先出
            node = queue.popleft()
            if node.left: queue.append(node.left)
            if node.right: queue.append(node.right)
            temp.append(node.val)
        # 一层访问完
        res.append(temp)
    return res

## 7. 树

- 深度优先搜索
    * 前序遍历、中序遍历、后序遍历
    * 路径长度，路径数量，最大路径
- 广度优先搜索
    * 序列化/反序列化二叉树
- 二叉搜索树
    * 树节点是排序的
    * 中序后继: 不断将root设置为比他大的节点
    * 所有大于等于节点的值之和: 按照右中左的顺序遍历
- 线段树
    * 日程表问题

常见题型: 树的高度，剪枝，序列化/反序列化，根都叶子结点路径长度，节点之和最大路径，二叉搜索树的中序后继，出现频率最高的K个数字

In [ ]:
# 二叉树剪枝，删除所有节点都为0的子树
def pruneTree(root):
    if not root: return root
    root.left = pruneTree(root.left)
    root.right = pruneTree(root.rihgt)
    if root.val == 0 and not root.left and not root.right:
        return None
    return root

# 序列化反序列化二叉树
def serialize(root):
    # 转化为bfs的字符串，没有的节点也要用None标记
    pass
def deserialize(root):
    # 按照bfs的顺序逐项添加
    pass

# 根到叶子的数字之和, 1->2->3表示123
def sumNumbers(root):
    def dfs(string, root):
        if not root: return
        string += str(root.val)
        if not root.left and not root.right:
            self.res += int(string)
        dfs(string, root.left)
        dfs(string, root.right)
    self.res = 0
    dfs("", root)
    return self.res

# 节点和为target的路径数量。
def pathSum(root, target):
    # 利用前缀和记录下从根节点到子节点路径上的各个和
    # 注意！！要将dic[0] = 1，以免出现合适路径从根节点开始不被加入的情况
    dic = defaultdict(int)
    res = 0
    def dfs(total, root):
        if not root: return
        total += root.val
        res += dic[target - total] 
        dic[total] += 1
        dfs(total, root.left)
        dfs(total, root.right)
        dic[total] -= 1
    dfs(0, root)
    return res

# 节点之和最大的路径。还蛮妙的
def maxPathSum(root):
    # https://leetcode-cn.com/problems/binary-tree-maximum-path-sum/solution/er-cha-shu-zhong-de-zui-da-lu-jing-he-by-leetcode-/
    def dfs(root):
        if not root: return 0
        # 如果子树为负数则贡献为0，不考虑
        left = max(0, dfs(root.left))
        right = max(0, dfs(root.right))
        # 以每一个节点为中节点，判断最长路径
        self.res = max(self.res, root.val + left + right)
        # 返回给上一层的是子树的最优路径
        return root.val + max(left, right)
    self.res = root.val
    dfs(root)
    return self.res




## 8. 堆

堆其实就是排序队列，原理可以用完全二叉树解释，可以用数组实现

- 大根堆(栈顶元素为最大值)，小根堆(栈顶元素为最小值)
- 默认小根堆，直接对数组使用heappush(list, val)来插入，用heappop(list)来弹出最小值
- 若想转化为大根堆可以将val取反再插入: heappush(list, -val)
- 常见题型: 找到最大的第K个数，出现频率最高的K个数，和最小的K个数。（就是维护一个大小固定的数组）

In [ ]:
# 数据流第K大元素
class KthLargest:
    def __init__(self, k, nums):
        self.heap = []
        self.k = k
        for num in nums:
            heappush(self.heap, num)
            if len(self.heap) > self.k:
                heappop(self.heap)

    def add(self, val: int) -> int:
        heappush(self.heap, val)
        if len(self.heap) > self.k:
            heappop(self.heap)
        return self.heap[0]

# 出现频率最高的K个数
def topKFrequent(nums, k):
    # 用堆来不断更新频率最高的k个数
    # 偷懒直接调函数
    return [i[0] for i in Counter(nums).most_common(k)]

# 和最小的K对数字
def KSmallestPairs(nums1, nums2, k):
    # 维护一个长度为K的堆
    res = []
    heap = []
    # 因为升序，只可能在两个数组的前k个元素中排列组合
    # 要找最小的K个元素，那么维护一个大根堆(栈顶元素是最大的)
    for i in range(min(k, len(nums1))):
        for j in range(min(k, len(nums2))):
            # 如果堆中还没有k个元素
            if len(heap) < k:
                heappush(heap, (-(nums1[i] + nums2[j]), nums1[i], nums2[j]))
            # 如果有k个元素了再判断是否要加入
            else:
                # 如果比栈顶元素还大不考虑，如果比栈顶元素小就可以加入
                if -(nums1[i] + nums2[j]) > heap[0][0]:
                    heappush(heap, (-(nums1[i] + nums2[j]), nums1[i], nums2[j]))
                    heappop(heap)
    # 完了之后得到的就是k个最小值
    for _, i, j in heap:
        res.append([i, j])
    return res

    # 或者全部可能找出来之后再排序
    temp = []
    for i in range(min(k, len(nums1))):
        for j in range(min(k, len(nums2))):
            temp.append(((nums1[i] + nums2[j]), nums1[i], nums2[j]))
    temp.sort(key = lambda x: x[0])
    return [[j, k] for i, j, k in temp[:k]]
            

## 9. 前缀树（字典树）
应用场景：自动补全、拼写检查、寻找词根、寻找相同前缀、相同后缀(反向插入)

主要操作：初始化，插入，查询字符串是否存在，查询相同前缀

In [2]:
class Node:
    def __init__(self):
        self.is_end = False
        self.child = {}


class Trie:
    def __init__(self):
        self.root = Node()
        
    def insert(self, word):
        head = self.root
        for c in word:
            if c not in head.child:
                head.child[c] = Node()
            head = head.child[c]
        head.is_end = True
        
    def search(self, word):
        head = self.root
        for c in word:
            if c not in head.child:
                return False
            head = head.child[c]
        return head.is_end
    
    
    def startswith(self, prefix):
        head = self.root
        for c in prefix:
            if c not in head.child:
                return False
            head = head.child[c]
        return True

    
test = Trie()
print(test.search("app"))
print(test.insert("app"))
print(test.search("app"))
print(test.startswith("a"))
print(test.startswith("b"))

False
None
True
True
False


## 10. 图

经常使用深度优先dfs、广度优先bfs、并查集等方法来解

- 深度优先搜索
    * 遍历所有组合方式
    * 标记所有与当前节点相连的节点
- 广度优先搜索
    * 一层层往外搜
    * 适合求最短距离
    * 标记所有与当前节点相连的节点
- 并查集
    * 求连通分量的个数 / 判断两个节点是否连通
    * 适合求图相关问题
    * 初始化:  parent = [for i in range(len(nums))] 
    * 关键函数:find(x)用来找节点最大的大哥；union(x, y)用来合并两个连通节点，使他们的大哥是同一个
    
常见问题: 岛屿最大面积，能否划分二分图，省份数量

In [13]:
# 岛屿最大面积
def maxAreaOfIsland(grid):
    m, n  = len(grid), len(grid[0])
    # 以某个点为起始点的岛屿，岛屿的面积是它加上下左右方向1的个数
    def bfs(i, j):
        if not (0 <= i < m and 0 <= j < n):
            return 0
        if not grid[i][j]: return 0
        grid[i][j] = 0
        return 1 + bfs(i+1, j) + bfs(i-1, j) + bfs(i, j+1) + bfs(i, j-1)

    res = 0
    for i in range(m):
        for j in range(n):
            res = max(res, bfs(i, j))
    return res
print("maxAreaOfIsland: ", maxAreaOfIsland([[0,0,1,0,0,0,0,1,0,0,0,0,0],[0,0,0,0,0,0,0,1,1,1,0,0,0],[0,1,1,0,1,0,0,0,0,0,0,0,0],[0,1,0,0,1,1,0,0,1,0,1,0,0],[0,1,0,0,1,1,0,0,1,1,1,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,1,1,1,0,0,0],[0,0,0,0,0,0,0,1,1,0,0,0,0]]))

# 二分图 (给定无向边列表，能否分割成两个独立的子集(每条边的两个顶点分别属于不同的集合))
def isBipatite(graph):
    # 用数组来记录各个节点，0为未访问，1为放进第一个数组，2为放进第2个数组
    colors = [0] * len(graph)
    queue = collections.deque()
    for i in range(len(graph)):
        # 没访问的放进第一个数组
        if colors[i] == 0:
            queue.append(i)
            colors[i] = 1
            while queue:
                # 标记为1的节点，和它相连的节点都应该标记为0或2；标记为2的节点，和它相连的节点都应该标记为0或1
                node = queue.popleft()
                target = (2 if colors[node] == 1 else 1)
                for j in graph[node]:
                    if colors[j] == 0:
                        colors[j] = target
                        queue.append(j)
                    elif colors[j] != target:
                        return False
    return True
print("isBipatite: ", isBipatite([[1,2,3],[0,2],[0,1,3],[0,2]]))

# 矩阵中的距离 (对于一个0-1矩阵，输出一个相同大小的矩阵记录对应元素到最近的0的距离)
def updateMatrix(mat):
    # 从0开始找1比从1开始找0方便。先找到所有0，做bfs逐渐增加步长
    m, n = len(mat), len(mat[0])
    res = [[0] * n for _ in range(m)]
    visited = [[0] * n for _ in range(m)]
    queue = collections.deque()
    for i in range(m):
        for j in range(n):
            if not mat[i][j]:
                visited[i][j] = 1
                queue.append((i, j, 0))
    while queue:
        i, j, k = queue.popleft()
        for a, b in [(i + 1, j), (i - 1, j), (i, j + 1), (i, j - 1)]:
            if (0 <= a < m and 0 <= b < n) and not visited[a][b]:
                visited[a][b] = 1
                res[a][b] = k + 1
                queue.append((a, b, k+1))
    return res
print("updateMatrix: ", updateMatrix([[0,0,0],[0,1,0],[1,1,1]]))

# 省份数量 (给定n个城市以及连通矩阵，isConnected[i][j]=1表示i与j相连，相连的城市构成一个省份，求省份的数量)
def findCircleNum(isConnected):
    # 适合用并查集找连通子集的数量 / 当然这道题bfs也很快，出现未访问的节点就把他相邻的城市都标记visited，统计有几个城市群
    parent = [i for i in range(len(isConnected))] 
    # 找这个城市所属的最大的大哥，最大的大哥指向他自己
    def find(x):
        while parent[x] != x:
            x = parent[x]
        return x
    # 如果两个城市连通，那么他们应该有共同的大哥
    def union(x, y):
        rootx = find(x)
        rooty = find(y)
        if rootx != rooty:
            parent[rootx] = rooty
    # 合并
    for i in range(len(isConnected)):
        for j in range(len(isConnected[i])):
            if isConnected[i][j] == 1:
                union(i, j)
    # 大哥指向他自己，统计大哥是自己的数量
    res = 0
    for i in range(len(isConnected)):
        if parent[i] == i:
            res += 1
    return res
print("findCircleNum: ", findCircleNum([[1,1,0],[1,1,0],[0,0,1]]))



maxAreaOfIsland:  6
isBipatite:  False
updateMatrix:  [[0, 0, 0], [0, 1, 0], [1, 2, 1]]
findCircleNum:  2
